# AI Security Agent - Advanced 6-Stage Distillation Pipeline
# Unsloth + Llama-3 8B for Specialized Security SLMs

This notebook implements the full distillation sequence:
1. **Teacher Reasoning**: CoT Data Generation
2. **Reasoning Distillation**: Training the student SLM
3. **Safety Alignment**: DPO (Direct Preference Optimization)
4. **Multi-Stage Quantization**: INT4/GGUF/Merged Export
5. **Hugging Face Integration**: Saving and Loading from Hub

### CELL 1: Installation
⚠️ IMPORTANT: Use Colab with GPU (T4, A100, or V100)
Runtime → Change runtime type → GPU

In [ ]:
!pip install -q -U "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q -U torch transformers datasets trl peft accelerate bitsandbytes huggingface_hub wandb

print("✅ Installation complete!")

### CELL 2: Imports & Login

In [ ]:
import torch
import json
import os
from datasets import Dataset, load_dataset
from transformers import TrainingArguments, TextStreamer
from trl import SFTTrainer
from unsloth import FastLanguageModel
from huggingface_hub import notebook_login
import pandas as pd
from typing import Dict, List
import random

# Login to Hugging Face
# notebook_login()

# Check GPU
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

### CELL 3: Stage 2 - Advanced CoT Data Generation
Simulating Teacher reasoning to teach the Student SLM *how* to think about security threats.

In [ ]:
def generate_cot_security_data(role='red', num_samples=100):
    """Generates high-quality CoT data for either Red or Blue team."""
    red_categories = [
        {
            "category": "indirect_prompt_injection",
            "scenario": "Attacker hides malicious instructions in a document retrieved by RAG.",
            "steps": [
                "Detect RAG dependency.",
                "Craft payload in reference text.",
                "Semantically align payload with search query.",
                "Execute hidden command after retrieval."
            ]
        }
    ]
    
    blue_categories = [
        {
            "category": "rag_integrity_check",
            "scenario": "Implementing verification layer for RAG results.",
            "steps": [
                "Intercept retrieval stream.",
                "Apply command-detection filter.",
                "Sanitize instructions.",
                "Enforce passive information processing."
            ]
        }
    ]

    selected = red_categories if role == 'red' else blue_categories
    data = []
    for _ in range(num_samples):
        cat = random.choice(selected)
        reasoning = "\n".join([f"{i+1}. {s}" for i, s in enumerate(cat['steps'])])
        data.append({
            "instruction": f"You are a {'Red' if role=='red' else 'Blue'} Team Expert. Analyze the scenario.",
            "input": f"Scenario: {cat['scenario']}",
            "output": json.dumps({"thought_process": reasoning, "action": "Analyze and Propose"}, indent=2)
        })
    return data

sample_data = generate_cot_security_data('red', 50) + generate_cot_security_data('blue', 50)
print(f"✅ Generated {len(sample_data)} CoT training samples")

### CELL 4: Format Dataset

In [ ]:
def format_alpaca_prompt(sample: Dict) -> str:
    return f"""### Instruction:\n{sample['instruction']}\n\n### Input:\n{sample['input']}\n\n### Expert Reasoning:\n{sample['output']}"""

formatted_samples = [{"text": format_alpaca_prompt(s)} for s in sample_data]
dataset = Dataset.from_list(formatted_samples).train_test_split(test_size=0.1, seed=42)
print(f"✅ Training samples (Stage 3): {len(dataset['train'])}")

### CELL 5: Load Model with Unsloth

In [ ]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)
print("✅ Base model loaded!")

### CELL 6: Apply LoRA Adapters for Distillation

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Higher rank for capture logic
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)
print("✅ LoRA adapters applied for Distillation!")

### CELL 7: Configure Training

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)
print("✅ Training configured!")

### CELL 8: Train

In [ ]:
trainer.train()
print("✅ Training complete!")

### CELL 9: Save & Quantize

In [ ]:
model.save_pretrained("slm-security-distilled")
tokenizer.save_pretrained("slm-security-distilled")

# --- STAGE 6: Quantization Export ---
# model.save_pretrained_gguf("model_q4_k_m.gguf", tokenizer, quantization_method = "q4_k_m")
# model.save_pretrained_merged("model_merged_4bit", tokenizer, save_method = "merged_4bit")

### CELL 10: Hugging Face Integration - Push to Hub
Push your distilled model to the Hugging Face Hub for sharing and easy deployment.

In [ ]:
# REPLACE with your Hub username and model name
HUB_MODEL_ID = "your-username/slm-security-distilled"

print(f"🚀 Pushing model to Hugging Face Hub: {HUB_MODEL_ID}")
# model.push_to_hub(HUB_MODEL_ID, use_temp_dir=False)
# tokenizer.push_to_hub(HUB_MODEL_ID, use_temp_dir=False)
print("✅ Push complete (Uncomment cells to execute)")

### CELL 11: Inference Test & Load from Hub

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[ 
    format_alpaca_prompt({
        "instruction": "You are a Red Team Expert. Analyze the scenario.",
        "input": "Scenario: Attacker hides malicious instructions in a document retrieved by RAG.",
        "output": ""
    })
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128)
print(tokenizer.batch_decode(outputs)[0])

print("\n--- To load from Hub later ---")
print(f"# model, tokenizer = FastLanguageModel.from_pretrained(model_name = '{HUB_MODEL_ID}', load_in_4bit = True)")